<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2048ca493483bd5b0242eb7680bbaef63bb78ac75ba4d1b4517a95dcabb99fb7
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-29 15:18:23
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: 1.22 L (0.84%)
Current PnL: -18.09 L (-11.85%)
CY Booked + Current PnL: -5.96 L (-3.91%)
-------------------
Total profit:  2.86 L
Total loss:  -20.95 L
-------------------
Total Booked + Current PnL: 21.02 L (16.73%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.3%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.16 L (58.28%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 9.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.16,76.0,X-LC,4.17,234608.0,24242.0,4528.0,1.14,11.52,1.93,13.68,37.0,5.35,1.62,29.95,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,111.95,66.0,M-SC,1.42,91314.0,-9463.0,9561.0,-2.67,-9.39,10.47,0.09,245.0,-0.99,0.63,19.97,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-15.53,75.0,H-MC,16.23,216581.0,36773.0,12583.0,-0.19,20.45,5.81,27.45,88.0,2.92,1.50,61.51,XR,NTT,BANKS
42,ITC,452.0,-36.23,67.0,X-LC,1.37,206260.0,6122.0,14768.0,0.93,3.06,7.16,10.44,4.0,0.41,1.43,8.79,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-9.84,40.0,X-MC,2.76,89130.0,-1548.0,16453.0,-0.20,-1.71,18.46,16.44,101.0,-0.09,0.62,18.12,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-12.21,69.0,X-LC,8.90,316129.0,287.0,112384.0,9.10,0.09,35.55,35.67,5.0,0.00,2.19,13.96,X40N,ATH,FMCG
82,VAIBHAVGBL,521.00,67.49,70.0,H-SC,3.75,145275.0,-37500.0,148573.0,7.36,-20.52,102.27,60.77,125.0,-0.25,1.00,30.12,XR,NTT,JEWELLERY
62,SAIL,228.00,53.17,67.0,M-MC,10.12,241838.0,16876.0,150786.0,6.27,7.50,62.35,74.53,192.0,0.11,1.67,42.17,XY24,BTT,STEEL
57,RECLTD,446.00,50.38,63.0,M-LC,5.46,211475.0,8305.0,33815.0,4.21,4.09,15.99,20.74,55.0,0.25,1.46,11.23,XY25,NTT,FINANCE
9,AWL,485.00,-63.00,65.0,X-MC,1.54,263102.0,-38649.0,200063.0,2.63,-12.81,76.04,53.50,116.0,-0.19,1.82,16.86,XY24,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,5211.76,-5.16,49.0,X-SC,0.42,107909.0,5905.0,38016.0,-3.29,5.79,35.23,43.06,122.0,0.16,0.75,23.90,X40N,ATH,MISC
79,TTKPRESTIG,770.00,111.95,66.0,M-SC,1.42,91314.0,-9463.0,9561.0,-2.67,-9.39,10.47,0.09,245.0,-0.99,0.63,19.97,OX40N,NTT,DURABLES
70,STARHEALTH,761.00,17.83,54.0,H-SC,12.65,262235.0,-18888.0,151755.0,-2.55,-6.72,57.87,47.26,144.0,-0.12,1.81,40.50,XY24,NTT,INSURANCE
19,COALINDIA,497.97,5.97,41.0,L-LC,2.39,141192.0,-9268.0,43064.0,-2.50,-6.16,30.50,22.46,167.0,-0.22,0.98,10.80,XY25,ATH,MINING
13,BATAINDIA,2096.00,-40.29,29.0,X-SC,4.73,89300.0,-39370.0,84665.0,-2.28,-30.60,94.81,35.20,219.0,-0.47,0.62,2.50,X40,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.0,-29.46,65.0,H-SC,16.15,201786.0,1832.0,95404.0,0.80,0.92,47.28,48.63,115.0,0.02,1.40,12.57,XR,NTT,MISC
87,WIPRO,420.0,-14.70,45.0,M-LC,5.73,150096.0,-849.0,110306.0,-0.12,-0.56,73.49,72.51,53.0,-0.01,1.04,5.43,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,111.95,66.0,M-SC,1.42,91314.0,-9463.0,9561.0,-2.67,-9.39,10.47,0.09,245.0,-0.99,0.63,19.97,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.06,56.0,H-SC,2.77,227565.0,-42102.0,78442.0,1.34,-15.61,34.47,13.47,138.0,-0.54,1.57,15.71,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.20,54.0,H-MC,6.76,105194.0,-26641.0,68871.0,-0.15,-20.21,65.47,32.03,98.0,-0.39,0.73,17.83,OX40N,NTT,IT
68,SIS,528.0,2046.63,48.0,H-SC,3.14,86636.0,-24396.0,48005.0,-0.64,-21.97,55.41,21.26,156.0,-0.51,0.60,17.05,OX40N,NTT,MISC
18,CERA,9475.0,-24.82,39.0,H-SC,2.64,137241.0,-38662.0,80684.0,-0.50,-21.98,58.79,23.89,149.0,-0.48,0.95,18.53,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.0,-15.53,75.0,H-MC,16.23,216581.0,36773.0,12583.0,-0.19,20.45,5.81,27.45,88.0,2.92,1.5,61.51,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,53.17,67.0,M-MC,10.12,241838.0,16876.0,150786.0,6.27,7.50,62.35,74.53,192.0,0.11,1.67,42.17,XY24,BTT,STEEL
57,RECLTD,446.00,50.38,63.0,M-LC,5.46,211475.0,8305.0,33815.0,4.21,4.09,15.99,20.74,55.0,0.25,1.46,11.23,XY25,NTT,FINANCE
32,HINDZINC,730.22,28.52,51.0,M-LC,11.36,209931.0,4855.0,107716.0,2.42,2.37,51.31,54.89,52.0,0.05,1.45,27.03,X5K,ATH,METALS
47,KPIGREEN,731.05,12.86,75.0,H-SC,5.41,134568.0,9271.0,49656.0,2.17,7.40,36.90,47.03,141.0,0.19,0.93,68.49,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,64.0,M-LC,9.41,166229.0,25389.0,19100.0,-0.59,18.03,11.49,31.59,66.0,1.33,1.15,49.29,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-15.53,75.0,H-MC,16.23,216581.0,36773.0,12583.0,-0.19,20.45,5.81,27.45,88.0,2.92,1.50,61.51,XR,NTT,BANKS
47,KPIGREEN,731.05,12.86,75.0,H-SC,5.41,134568.0,9271.0,49656.0,2.17,7.40,36.90,47.03,141.0,0.19,0.93,68.49,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.39,130005.0,15207.0,78835.0,2.31,13.25,60.64,81.92,223.0,0.19,0.90,53.91,XR,NTT,DURABLES
37,INDIAMART,4810.62,-52.77,59.0,H-SC,6.81,131663.0,8327.0,123302.0,0.15,6.75,93.65,106.72,119.0,0.07,0.91,31.71,AR,ATH,MISC
32,HINDZINC,730.22,28.52,51.0,M-LC,11.36,209931.0,4855.0,107716.0,2.42,2.37,51.31,54.89,52.0,0.05,1.45,27.03,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.03,5.28,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.29,29.0,X-SC,4.73,89300.0,-39370.0,84665.0,-2.28,-30.60,94.81,35.20,219.0,-0.47,0.62,2.50,X40,NTT,FOOTWEAR
45,JSWINFRA,342.00,-20.48,37.0,X-MC,5.33,199766.0,-581.0,31084.0,0.34,-0.29,15.56,15.23,178.0,-0.02,1.38,31.87,X40N,NTT,REALTY
24,DMART,5391.45,-17.07,37.0,X-LC,9.88,101460.0,-1126.0,27932.0,0.16,-1.10,27.53,26.13,38.0,-0.04,0.70,24.19,X40N,ATH,FMCG
31,HINDUNILVR,2922.00,-11.49,40.0,X-LC,7.61,248750.0,-8313.0,43457.0,-0.38,-3.23,17.47,13.67,24.0,-0.19,1.72,16.52,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.18,49.0,H-LC,1.06,157230.0,-28865.0,76351.0,1.12,-15.51,48.56,25.51,15.0,-0.38,1.09,15.87,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.23,67.0,X-LC,1.37,206260.0,6122.0,14768.0,0.93,3.06,7.16,10.44,4.0,0.41,1.43,8.79,X40,NTT,FMCG
53,PGHH,17907.65,-30.28,40.0,X-MC,2.58,201900.0,1080.0,66708.0,0.46,0.54,33.04,33.76,80.0,0.02,1.40,5.76,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.89,42.0,X-MC,3.08,310926.0,-17065.0,123593.0,-0.80,-5.20,39.75,32.48,92.0,-0.14,2.15,5.63,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,1.30,53.0,X-MC,3.90,212640.0,20898.0,16862.0,-0.76,10.90,7.93,19.69,99.0,1.24,1.47,18.44,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.29,29.0,X-SC,4.73,89300.0,-39370.0,84665.0,-2.28,-30.60,94.81,35.20,219.0,-0.47,0.62,2.50,X40,NTT,FOOTWEAR
10,BAJAJHFL,181.50,-13.84,50.0,X-MC,5.91,181614.0,-18892.0,116051.0,1.59,-9.42,63.90,48.45,90.0,-0.16,1.26,4.73,X40N,ATH,FINANCE
52,PAGEIND,51769.93,-26.81,45.0,X-MC,6.77,82740.0,-770.0,20801.0,1.11,-0.92,25.14,23.98,82.0,-0.04,0.57,4.84,X40,ATH,APPARELS
21,COLPAL,3767.14,-2.79,51.0,X-MC,7.84,226710.0,-36655.0,150014.0,1.54,-13.92,66.17,43.04,84.0,-0.24,1.57,5.25,XY25,ATH,FMCG
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.03,5.28,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-44.35,54.0,X-SC,38.16,55089.0,-9917.0,160843.0,2.15,-15.26,291.97,232.18,198.0,-0.06,0.38,7.45,XY24,NTT,MISC
52,PAGEIND,51769.93,-26.81,45.0,X-MC,6.77,82740.0,-770.0,20801.0,1.11,-0.92,25.14,23.98,82.0,-0.04,0.57,4.84,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.20,47.0,X-SC,4.81,82842.0,-62318.0,138247.0,1.43,-42.93,166.88,52.31,136.0,-0.45,0.57,10.89,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.84,40.0,X-MC,2.76,89130.0,-1548.0,16453.0,-0.20,-1.71,18.46,16.44,101.0,-0.09,0.62,18.12,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.29,29.0,X-SC,4.73,89300.0,-39370.0,84665.0,-2.28,-30.60,94.81,35.20,219.0,-0.47,0.62,2.50,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.04,55.0,X-LC,11.31,287339.0,-58617.0,125309.0,-0.04,-16.94,43.61,19.28,1.0,-0.47,1.99,6.22,X40,ATH,IT
40,INFY,2275.00,-17.80,58.0,X-LC,6.56,319543.0,6655.0,165044.0,-0.29,2.13,51.65,54.87,3.0,0.04,2.21,10.59,X40,BTT,IT
42,ITC,452.00,-36.23,67.0,X-LC,1.37,206260.0,6122.0,14768.0,0.93,3.06,7.16,10.44,4.0,0.41,1.43,8.79,X40,NTT,FMCG
84,VBL,671.64,-12.21,69.0,X-LC,8.90,316129.0,287.0,112384.0,9.10,0.09,35.55,35.67,5.0,0.00,2.19,13.96,X40N,ATH,FMCG
1,ABB,7934.00,-39.28,57.0,H-LC,5.79,253632.0,-7987.0,127196.0,2.10,-3.05,50.15,45.57,7.0,-0.06,1.75,7.92,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7038.89,59.0,L-SC,7.59,78310.0,-15500.0,92116.0,-1.27,-16.52,117.63,81.67,269.0,-0.17,0.54,51.65,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.92,81216.0,-32333.0,72388.0,-0.68,-28.47,89.13,35.27,268.0,-0.45,0.56,98.78,XR,NTT,HOTELS
50,MASFIN,398.61,-20.05,48.0,H-SC,9.16,91095.0,-6885.0,28485.0,0.65,-7.03,31.27,22.05,152.0,-0.24,0.63,32.41,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,111.95,66.0,M-SC,1.42,91314.0,-9463.0,9561.0,-2.67,-9.39,10.47,0.09,245.0,-0.99,0.63,19.97,OX40N,NTT,DURABLES
86,WHIRLPOOL,2270.00,-37.26,66.0,M-SC,3.39,130005.0,15207.0,78835.0,2.31,13.25,60.64,81.92,223.0,0.19,0.90,53.91,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,KPIGREEN,731.05,12.86,75.0,H-SC,5.41,134568.0,9271.0,49656.0,2.17,7.40,36.90,47.03,141.0,0.19,0.93,68.49,MH,ATH,POWER
63,SAMMAANCAP,326.00,-195.45,73.0,M-SC,29.63,165699.0,16479.0,127704.0,-1.68,11.04,77.07,96.62,208.0,0.13,1.15,79.32,XY25,NTT,FINANCE
69,SONACOMS,806.63,-32.06,71.0,M-SC,7.97,84525.0,-16634.0,56632.0,-0.28,-16.44,67.00,39.54,202.0,-0.29,0.58,19.19,AR,ATH,AUTO
12,BANKINDIA,149.00,-15.53,75.0,H-MC,16.23,216581.0,36773.0,12583.0,-0.19,20.45,5.81,27.45,88.0,2.92,1.50,61.51,XR,NTT,BANKS
5,ANGELONE,3033.00,16.33,63.0,X-SC,5.43,205681.0,14675.0,43028.0,-1.17,7.68,20.92,30.21,157.0,0.34,1.42,30.17,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,44.04
2,50,74.95


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.02
LC    32.35
MC    24.62
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.08
X40      16.20
XY25     12.03
XR       11.42
X40N     10.69
AR        7.84
OX40N     6.51
X200      1.76
X5K       1.45
SR        1.08
MH        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.45
X-LC    21.67
X-MC    18.75
M-SC    12.63
M-LC     5.10
H-LC     4.60
X-SC     4.51
H-MC     3.88
M-MC     1.67
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.59,-2.19,35.54
IT,12.48,-16.48,78.95
FINANCE,12.12,-7.87,56.77
MISC,6.86,-16.05,80.35
ELECTRICAL,5.76,-8.37,46.78
BANKS,5.66,-4.65,58.76
PAINTS,5.51,-15.77,33.02
INSURANCE,3.79,-0.91,38.53
AUTO,2.75,-44.14,104.30


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3221765.0,22
XR,1232545.0,14
AR,1155152.0,9
X40,815608.0,12
X40N,613923.0,9
OX40N,542372.0,9
XY25,443450.0,8
SR,265012.0,2
X5K,107716.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3295426.0,24
M-SC,1215783.0,15
X-MC,1168477.0,14
X-LC,1004263.0,13
X-SC,527774.0,6
H-LC,272405.0,3
M-LC,270937.0,4
H-MC,257178.0,3
L-SC,253356.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1146020.0      6
           AR         828766.0      5
           XR         762677.0      7
M-SC       XY24       698019.0      6
X-MC       XY24       567488.0      4
X-LC       X40        443965.0      5
           XY24       303785.0      2
X-MC       X40        286978.0      6
X-SC       X40N       282266.0      4
H-SC       SR         265012.0      2
           OX40N      243295.0      3
H-LC       AR         203547.0      2
X-LC       X40N       184522.0      3
H-MC       XY24       175724.0      1
X-MC       XY25       166876.0      2
L-SC       XR         164504.0      2
X-SC       XY24       160843.0      1
M-MC       XY24       150786.0      1
X-MC       X40N       147135.0      2
M-SC       OX40N      141354.0      4
           XY25       127704.0      1
           XR         125867.0      2
           AR         122839.0      2
M-LC       XR         110306.0      1
           X5K        107716.0      1
L-SC       OX40N       88852.0      1
X-SC       X40         84665.0      1
X-LC       XY25        71991.0      3
H-MC       OX40N       68871.0      1
H-LC       X200        68858.0      1
L-MC       XR          56608.0      1
H-SC       MH          49656.0      1
L-LC       XY25        43064.0      1
M-LC       XY25        33815.0      1
           XY24        19100.0      1
H-MC       XR          12583.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
